# Trading and backtesting using Backtrader framework

An example how to filter and plot candles for pair data

We will

* Have $1000 starting balance

* Create a trading pair universe consisting of USDC, USDT and ETH quoted pairs on Sushiswap

* Randomly go all into a new pair, having more thank $500k trading volume

* Pay some attention to make random behavior deterministic

## Creating trading universe

First let's import libraries and initialise our dataset client.

In [13]:

import pandas as pd

from capitalgram.chain import ChainId
from capitalgram.pair import PairUniverse

try:
    import capitalgram
except ImportError:
    !pip install -e git+https://github.com/miohtama/capitalgram-onchain-dex-quant-data.git#egg=capitalgram
    import site
    site.main()

from capitalgram.client import Capitalgram

capitalgram = Capitalgram.create_jupyter_client()

Started Capitalgram in Jupyter notebook environment, configuration is stored in /Users/moo/.capitalgram


Let's create a pair universe for Sushi. [See full example](https://docs.capitalgram.com/examples/pairs.html).
We will create a dataset of 4h candles that trade on Sushiswap on Ethereum.

In [14]:
# Decompress the pair dataset to Python map
columnar_pair_table = capitalgram.fetch_pair_universe()

# Exchange map data is so small it does not need any decompression
exchange_universe = capitalgram.fetch_exchange_universe()

# Convert PyArrow table to Pandas format to continue working on it
all_pairs_dataframe = columnar_pair_table.to_pandas()

# Filter down to pairs that only trade on Sushiswap
sushi_swap = exchange_universe.get_by_name_and_chain(ChainId.ethereum, "sushiswap")
sushi_pairs: pd.DataFrame = all_pairs_dataframe.loc[all_pairs_dataframe['exchange_id'] == sushi_swap.exchange_id]

# Create a Python set of pair ids
wanted_pair_ids = sushi_pairs["pair_id"]

print(f"Sushiswap on Ethereum has {len(wanted_pair_ids)} pairs")

Sushiswap on Ethereum has 1308 pairs


Get daily candles and filter them against our wanted pair set.

In [15]:
from capitalgram.candle import CandleBucket

# Get daily candles as Pandas DataFrame
all_candles = capitalgram.fetch_all_candles(CandleBucket.h24).to_pandas()
sushi_candles = all_candles.loc[all_candles["pair_id"].isin(wanted_pair_ids)]

print(f"Out candle universe size is {len(sushi_candles)} candles")

Out candle universe size is 58627 candles


## Creating the first Backtrader strategy

[See the Backtrader quickstart tutorial](https://www.backtrader.com/docu/quickstart/quickstart/).